In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
data = pd.read_parquet("../fhvhv_tripdata_2021-01.parquet", engine="pyarrow")
print(data.head())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B02682               B02682   
1            HV0003               B02682               B02682   
2            HV0003               B02764               B02764   
3            HV0003               B02764               B02764   
4            HV0003               B02764               B02764   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2021-01-01 00:28:09 2021-01-01 00:31:42 2021-01-01 00:33:44   
1 2021-01-01 00:45:56 2021-01-01 00:55:19 2021-01-01 00:55:19   
2 2021-01-01 00:21:15 2021-01-01 00:22:41 2021-01-01 00:23:56   
3 2021-01-01 00:39:12 2021-01-01 00:42:37 2021-01-01 00:42:51   
4 2021-01-01 00:46:11 2021-01-01 00:47:17 2021-01-01 00:48:14   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2021-01-01 00:49:07           230           166        5.26  ...       1.98   
1 2021-01-01 01:18:21           152           167       

In [3]:
data['driver_pay_per_mile'] = data['driver_pay'] / data['trip_miles']
data['trip_duration'] = (data['dropoff_datetime'] - data['pickup_datetime']).dt.total_seconds()/60
data['driver_pay_per_minute'] = data['driver_pay'] / data['trip_duration']


In [4]:
# create a model to use all the features to predict driver_pay
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(data.columns)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# split the data into training and testing
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)

train_labels = train_data.pop('driver_pay')
test_labels = test_data.pop('driver_pay')

model.fit(train_data, train_labels, epochs=10)

test_loss = model.evaluate(test_data, test_labels)

print(f"Test loss: {test_loss}")


c:\Users\bhavy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
